In [ ]:
!pip install scikit-learn
!pip install rouge
!pip install nltk
from rouge import Rouge 
import nltk
import nltk.translate.bleu_score as bleu
nltk.download('punkt')
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
text_len_l = []
text_len_s = []
text_len_w = []
sum_len_w = []
sum_len_l = []
######
rouge1_f1_5 = []
rouge1_f1_10 = []
rouge1_f1_15 = []
rouge1_f1_20 = []

rouge2_f1_5 = []
rouge2_f1_10 = []
rouge2_f1_15 = []
rouge2_f1_20 = []

rougel_f1_5 = []
rougel_f1_10 = []
rougel_f1_15 = []
rougel_f1_20 = []

bleu_5 = []
bleu_10 = []
bleu_15 = []
bleu_20 = []

sum_list = []

In [ ]:
def summary_to_sentences(summary):
    # Разделяем резюме на предложения, используя символ "." в качестве разделителя
    sentences = re.split("[.!?]", summary)
    
    # Преобразуем каждое предложение в список слов
    sentence_lists = [sentence.split() for sentence in sentences]
    
    return sentence_lists

def paragraph_to_wordlist(paragraph):
    # Разделим абзац на слова, используя пробел в качестве разделителя
    words = paragraph.split()
    return words

def listToString(s):
 
    # инициализируйте пустую строку
    str1 = ""
 
    # перемещение по строке
    for ele in s:
        str1 += ele
 
    # возвращаемая строка
    return str1

In [ ]:
import os,glob,re
path = "/content/drive/My Drive/texts/London J"
for filename in glob.glob(os.path.join(path, '*.txt')):
    with open(os.path.join(os.getcwd(), filename), 'r') as f: 
      text = f.read().replace('\n', '').replace('\r', '')

    num_sentences = 5
    while num_sentences < 21:
        sentences = re.split("[.!?]",text)
        text_len_l.append(len(text))
        text_len_s.append(len(sentences))
        text_len_w.append(len(listToString(sentences).split()))
        vectorizer = TfidfVectorizer(stop_words=list('russian'))
        X = vectorizer.fit_transform([text])
        lsa = TruncatedSVD(num_sentences, algorithm='randomized', n_iter=1000) 
        lsa.fit(X)
        sentences = re.split("[.!?]",text)
        important_sentences = np.argsort(np.abs(lsa.components_[0]))[::-1]

        # Убедимся, что индексы в important_sentences находятся в пределах допустимых индексов для предложений
        valid_indices = [i for i in important_sentences if i < len(sentences)]

        # Извлечемм два наиболее важных предложения на основе допустимых индексов
        summary_sentences = [sentences[i].strip() for i in valid_indices[:num_sentences]]

        # Если допустимых индексов недостаточно, дополними сводку пустыми строками
        while len(summary_sentences) < 2:
            summary_sentences.append('')

        summary = '. '.join(summary_sentences) + '.'

        rouge = Rouge()
        scores = rouge.get_scores(summary, text)
        sum_len_l.append(len(summary))
        sum_len_w.append(len(summary.split()))
        from nltk.translate.bleu_score import sentence_bleu
        reference_paragraph = text
        reference_summary = summary_to_sentences(reference_paragraph)
        predicted_paragraph = summary
        predicted_summary = paragraph_to_wordlist(predicted_paragraph)

        score = sentence_bleu(reference_summary, predicted_summary)
        match num_sentences:
          case 5:
            rouge1_f1_5.append(scores[0]['rouge-1']['f'])
            bleu_5.append(score)
          case 10:
            rouge1_f1_10.append(scores[0]['rouge-1']['f'])
            bleu_10.append(score) 
          case 15:
            rouge1_f1_15.append(scores[0]['rouge-1']['f'])
            bleu_15.append(score)
          case 20:
            rouge1_f1_20.append(scores[0]['rouge-1']['f'])
            bleu_20.append(score)    

        sum_list.append(summary)
        num_sentences = num_sentences + 5

In [ ]:
import csv

fields1 = ['text_len_l', 'text_len_s','text_len_w', 'sum_len_l','sum_len_w']
rows1 = [text_len_l, text_len_s, text_len_w, sum_len_l, sum_len_w]

fields2 = ['rouge1_f1_5','bleu5','rouge1_f1_10','bleu10','rouge1_f1_15','bleu_15','rouge1_f1_20','bleu_20','sum_list']
rows2 = [rouge1_f1_5,bleu_5,rouge1_f1_10,bleu_10,rouge1_f1_15,bleu_15,rouge1_f1_20,bleu_20, sum_list]

from itertools import zip_longest
export_data = zip_longest(*rows1, fillvalue = '')
with open('len_short_LSA.csv', 'w', encoding="utf-8", newline='') as myfile:
      wr = csv.writer(myfile)
      wr.writerow((fields1))
      wr.writerows(export_data)
myfile.close()


export_data = zip_longest(*rows2, fillvalue = '')
with open('metric_short_LSA.csv', 'w', encoding="utf-8", newline='') as myfile:
      wr = csv.writer(myfile)
      wr.writerow((fields2))
      wr.writerows(export_data)
myfile.close()